In [1]:
import importlib
cim_profile = 'rc4_2021'
cim = importlib.import_module('cimlab.data_profile.' + cim_profile)


In [2]:
from cimlab.loaders import Parameter, ConnectionParameters
from cimlab.loaders.blazegraph.blazegraph import BlazegraphConnection
from cimlab.loaders.gridappsd import GridappsdConnection, get_topology_response
from cimlab.models.distributed_model import DistributedModel
# import cim.data_profile as cim
import json


In [3]:
# # Blazegraph connection for running inside the container
# params = ConnectionParameters([Parameter(key="url", value="http://blazegraph:8080/bigdata/namespace/kb/sparql")])
# bg = BlazegraphConnection(params)

In [4]:
# Blazegraph connection for running outside the container
params = ConnectionParameters([Parameter(key="url", value="http://localhost:8889/bigdata/namespace/kb/sparql")])
bg = BlazegraphConnection(params, 'rc4_2021')

In [5]:
#feeder_mrid = "_C1C3E687-6FFD-C753-582B-632A27E28507"  # 123 bus
# feeder_mrid = "_49AD8E07-3BF9-A4E2-CB8F-C3722F837B62"  # 13 bus
feeder_mrid = "_5B816B93-7A5F-B64C-8460-47C17D6E4B0F" # 13 bus asets
#feeder_mrid = "_4F76A5F9-271D-9EB8-5E31-AA362D86F2C3"  # 8500 node
#feeder_mrid = "_67AB291F-DCCD-31B7-B499-338206B9828F" # J1
#feeder_mrid = "_9CE150A8-8CC5-A0F9-B67E-BBD8C79D3095"  # R2 12.47 3
#feeder_mrid = "_EE71F6C9-56F0-4167-A14E-7F4C71F10EAA" #9500 node

In [6]:
feeder = cim.Feeder(mRID=feeder_mrid)

In [7]:
# from gridappsd import GridAPPSD
from topology_processor import DistributedTopology

gapps = GridappsdConnection(feeder_mrid)
Topology = DistributedTopology(gapps, feeder_mrid)
topology_response = Topology.create_switch_areas(feeder_mrid)
# topology_response = get_topology_response(feeder_mrid)
topology_response = json.loads(topology_response)
# with open('topo_message.json') as topo_file:
#     topology_response = json.load(topo_file)


In [8]:
# topology_response = json.loads(topology_response)

In [9]:
network = DistributedModel(connection=bg, feeder=feeder, topology=topology_response)

In [10]:
switch_area = network.switch_areas[0]

# Get phase, bus info about ACLineSegments
switch_area.get_all_attributes(cim.ACLineSegment)
switch_area.get_all_attributes(cim.ACLineSegmentPhase)
switch_area.get_all_attributes(cim.Terminal)

line_ids = list(switch_area.typed_catalog[cim.ACLineSegment].keys())
for line_id in line_ids:
    print('line mrid: ',line_id)
    print('line name:', switch_area.typed_catalog[cim.ACLineSegment][line_id].name)
    print('bus 1: ', switch_area.typed_catalog[cim.ACLineSegment][line_id].Terminals[0].ConnectivityNode.mRID)
    print('bus 2: ', switch_area.typed_catalog[cim.ACLineSegment][line_id].Terminals[1].ConnectivityNode.mRID)
    for line_phs in switch_area.typed_catalog[cim.ACLineSegment][line_id].ACLineSegmentPhases:
        print('phase ', line_phs.phase, ': ', line_phs.mRID)
    

line mrid:  _6EBB2188-CBE1-4FEC-9406-83DE8E61F9CB
line name: 692675
bus 1:  _04006A64-109D-499D-8496-1B55907C1570
bus 2:  _CB29A4D1-EE08-4AAF-82F6-3C178009EBC0
phase  A :  _35FF14DC-7F66-44F6-A767-32F3D382C173
phase  B :  _C3AAFFD5-8C98-4DB5-8E6C-DC99E345D13C
phase  C :  _C7529946-755B-4322-950A-AE9FD9D9F1F6


In [11]:
switch_area = network.switch_areas[0]

switch_area.get_all_attributes(cim.EnergyConsumer)
switch_area.get_all_attributes(cim.EnergyConsumerPhase)

load_ids = list(switch_area.typed_catalog[cim.EnergyConsumer].keys())

for load in load_ids:
    print('p = ', switch_area.typed_catalog[cim.EnergyConsumer][load].p)
    print('q = ', switch_area.typed_catalog[cim.EnergyConsumer][load].q)
    for load_phs in switch_area.typed_catalog[cim.EnergyConsumer][load].EnergyConsumerPhase:
        print('phases: ', load_phs.phase)

p =  290000
q =  212000
phases:  C
p =  170000
q =  151000
phases:  C
p =  485000
q =  190000
phases:  A
p =  68000
q =  60000
phases:  B


In [12]:
network.get_all_attributes(cim.TransformerTank)
network.get_all_attributes(cim.TransformerTankEnd)
network.get_all_attributes(cim.TransformerTankInfo)
network.get_all_attributes(cim.TransformerEndInfo)
network.get_all_attributes(cim.ShortCircuitTest)
network.get_all_attributes(cim.NoLoadTest)

In [15]:
network.get_all_attributes(cim.ACLineSegment)
network.get_all_attributes(cim.ACLineSegmentPhase)
network.get_all_attributes(cim.PerLengthPhaseImpedance)
network.get_all_attributes(cim.PhaseImpedanceData)

network.get_all_attributes(cim.WireSpacingInfo)
network.get_all_attributes(cim.WirePosition)
network.get_all_attributes(cim.OverheadWireInfo)
network.get_all_attributes(cim.ConcentricNeutralCableInfo)
network.get_all_attributes(cim.TapeShieldCableInfo)

network.get_all_attributes(cim.Terminal) #bus info

In [17]:
line_ids = list(switch_area.typed_catalog[cim.ACLineSegment].keys())
for line_id in line_ids:
    print('line mrid: ',line_id)
    print('line name:', switch_area.typed_catalog[cim.ACLineSegment][line_id].name)
    print('bus 1: ', switch_area.typed_catalog[cim.ACLineSegment][line_id].Terminals[0].ConnectivityNode.mRID)
    print('bus 2: ', switch_area.typed_catalog[cim.ACLineSegment][line_id].Terminals[1].ConnectivityNode.mRID)
    for line_phs in switch_area.typed_catalog[cim.ACLineSegment][line_id].ACLineSegmentPhases:
        print('phase ', line_phs.phase, ': ', line_phs.mRID)
    if switch_area.typed_catalog[cim.ACLineSegment][line_id].PerLengthImpedance is not None:
        for impedance in switch_area.typed_catalog[cim.ACLineSegment][line_id].PerLengthImpedance.PhaseImpedanceData:
            print('row:', impedance.row, 'col:', impedance.column, 'r:', impedance.r, 'x:', impedance.x, 'b:', impedance.b)
    if switch_area.typed_catalog[cim.ACLineSegment][line_id].WireSpacingInfo is not None:
        for impedance in switch_area.typed_catalog[cim.ACLineSegment][line_id].PerLengthImpedance.PhaseImpedanceData:
            print('row:', impedance.row, 'col:', impedance.column, 'r:', impedance.r, 'x:', impedance.x, 'b:', impedance.b)

line mrid:  _6EBB2188-CBE1-4FEC-9406-83DE8E61F9CB
line name: 692675
bus 1:  _04006A64-109D-499D-8496-1B55907C1570
bus 2:  _CB29A4D1-EE08-4AAF-82F6-3C178009EBC0
phase  A :  _35FF14DC-7F66-44F6-A767-32F3D382C173
phase  B :  _C3AAFFD5-8C98-4DB5-8E6C-DC99E345D13C
phase  C :  _C7529946-755B-4322-950A-AE9FD9D9F1F6


In [ ]:
print(switch_area.get_attributes_query(cim.ConcentricNeutralCableInfo))

In [ ]:
json.loads(switch_area.__dumps__(cim.PhaseImpedanceData))

In [ ]:
secondary_area = network.switch_areas[4].secondary_areas[0]
secondary_area.get_all_attributes(cim.PowerElectronicsConnection)
secondary_area.get_all_attributes(cim.PowerElectronicsConnectionPhase)

pec_ids = list(secondary_area.typed_catalog[cim.PowerElectronicsConnection])

for pec in pec_ids:
    print('p = ', secondary_area.typed_catalog[cim.PowerElectronicsConnection][pec].p)
    print('q = ', secondary_area.typed_catalog[cim.PowerElectronicsConnection][pec].q)
    for pec_phs in secondary_area.typed_catalog[cim.PowerElectronicsConnection][pec].PowerElectronicsConnectionPhases:
        print('phase ', pec_phs.phase, ': ', pec_phs.mRID)

In [20]:
network.typed_catalog[cim.ACLineSegment][line_id].WireSpacingInfo

WireSpacingInfo(mRID='_48A5C17E-B098-446D-B26F-2D1B198ED260', aliasName=None, description=None, name='606', Names=[], Assets=[], CatalogAssetType=None, PowerSystemResources=[], ProductAssetModel=None, isCable='true', phaseWireCount='1', phaseWireSpacing='0', usage=None, ACLineSegments=[ACLineSegment(mRID='_6EBB2188-CBE1-4FEC-9406-83DE8E61F9CB', aliasName=None, description=None, name='692675', Names=[], AssetDatasheet=None, Assets=[], ConfigurationEvent=[], Controls=[], Location=Location(mRID='_6D36746B-C083-4CE2-87DD-256D6C668613', aliasName=None, description=None, name='692675_Loc', Names=[], direction=None, geoInfoReference=None, type=None, Assets=[], ConfigurationEvents=[], CoordinateSystem=None, Crew=[], electronicAddress=None, EnvironmentalLocationKind=[], Fault=[], Hazards=[], mainAddress=None, Measurements=[], phone1=None, phone2=None, PositionPoints=[], PowerSystemResources=[], secondaryAddress=None, status=None), Measurements=[Analog(mRID='_18b33a13-c766-48f2-af7a-09027fd38693

In [19]:
cim.ACLineSegment(())

ACLineSegment(mRID=(), aliasName=None, description=None, name=None, Names=[], AssetDatasheet=None, Assets=[], ConfigurationEvent=[], Controls=[], Location=None, Measurements=[], OperatingShare=[], PSRType=None, ReportingGroup=[], aggregate=None, inService=None, networkAnalysisEnabled=None, normallyInService=None, AdditionalEquipmentContainer=[], EquipmentContainer=None, Faults=[], OperationalLimitSet=[], UsagePoints=None, BaseVoltage=None, SvStatus=[], Terminals=[], length=None, b0ch=None, bch=None, g0ch=None, gch=None, r=None, r0=None, shortCircuitEndTemperature=None, x=None, x0=None, ACLineSegmentPhases=[], Clamp=[], Cut=[], LineFaults=[], ParallelLineSegment=None, PerLengthImpedance=None, WireSpacingInfo=None)

In [ ]:
values = ['_0649C1EF-E39F-4B46-AD80-D3C180E4562E']
message = bg.sparql.get_class_type_sparql(feeder_mrid, values)
output = bg.execute(message)
out = output['results']['bindings']
if values and not out:
    print('yes')